In [1]:
import gym
import numpy as np

import time

In [2]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [3]:
import matplotlib.pyplot as plt

In [4]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable
if find_executable('latex'):
    plt.rc('text', usetex=True)

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

In [5]:
import torch

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
import models.rnn as rnns
import models.mlp as mlps
import models.linear as linears
import control.agents as agents
import control.environments as env

In [8]:
from utils.notifications import Slack

In [9]:
import copy

# Check the activated features

In [14]:
model = linears.Linear(input_dimension=128, n_actions=4)
optimiser = torch.optim.Adam(model.parameters(), lr=.005)
agent = agents.DQNAgent(model, optimiser, gamma=.99, temperature=1, algorithm='expsarsa', n_actions=4)

environment = env.Environment(
    environment=gym.make('Breakout-ram-v0'), 
    agent=agent,
    verbose=True,
    max_steps=300,
    capacity=5000,
    representation_method='vector'
)

/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [16]:
m, M = environment.bytes_evolution_range(n_episodes_exploration=1000, n_episodes_evaluation=100)

activated_bytes = np.nonzero(M - m)[0]
min_bytes = m[activated_bytes]
max_bytes = M[activated_bytes]
bytes_range = max_bytes - min_bytes

KeyboardInterrupt: 

In [12]:
print(("{} activated bytes (coded on {} integers in total): " + ", ".join([str(x) for x in activated_bytes])).format(len(activated_bytes), len(activated_bytes) + sum(bytes_range)))
print()
print("Min bytes: " + ", ".join([str(x) for x in min_bytes]))
print()
print("Max bytes: " + ", ".join([str(x) for x in max_bytes]))
print()
print("Bytes ranges: " + ", ".join([str(x) for x in bytes_range]))


14 activated bytes (coded on 1376 integers in total): 49, 57, 71, 74, 75, 86, 90, 94, 95, 99, 101, 103, 105, 119

Min bytes: 225, 4, 1, 2, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0

Max bytes: 240, 5, 6, 8, 13, 25, 100, 228, 71, 199, 208, 1, 255, 255

Bytes ranges: 15, 1, 5, 6, 13, 5, 100, 228, 71, 199, 208, 1, 255, 255


# Check the dimension of the representation space

## For Environment (vector or tiling)

In [13]:
environment = env.Environment(
    environment=gym.make('Breakout-ram-v0'), 
    agent=None,
    verbose=True,
    max_steps=300,
    capacity=5000,
    n_tilings=5,
    n_bins=20
)

In [14]:
for representation_method in ['vector', 'tiling']:
    environment.representation_method = representation_method
    print("{}: {}".format(representation_method, environment.get_input_dimension()))

(128,)
(12800,)


## For OverSimplifiedEnvironment (vector, tiling, 1 hot encoding or mixed)

In [15]:
environment = env.OverSimplifiedEnvironment(
    environment=gym.make('Breakout-ram-v0'), 
    agent=None,
    verbose=True,
    max_steps=300,
    capacity=5000,
    mixed_threshold=10,
    max_tiling=5
)

### Check the current environment:

In [16]:
activated_bytes = environment.features
min_bytes = environment.min_range
max_bytes = environment.max_range
bytes_range = max_bytes - min_bytes
tilings = environment.tilings

In [19]:
print(("{} activated bytes (coded on {} integers in total): " + ", ".join([str(x) for x in activated_bytes])).format(len(activated_bytes), len(activated_bytes) + sum(bytes_range)))
print()
print("Min bytes: " + ", ".join([str(x) for x in min_bytes]))
print()
print("Max bytes: " + ", ".join([str(x) for x in max_bytes]))
print()
print("Bytes ranges: " + ", ".join([str(x) for x in bytes_range]))
print()
print("Tilings: {}".format(tilings))


26 activated bytes (coded on 3074 integers in total): 30, 49, 57, 70, 71, 72, 74, 75, 77, 84, 86, 90, 91, 94, 95, 96, 99, 100, 101, 103, 105, 106, 107, 119, 121, 122

Min bytes: 0, 225, 4, 0, 1, 55, 2, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150

Max bytes: 192, 240, 5, 184, 6, 191, 8, 13, 1, 5, 25, 255, 3, 228, 135, 28, 232, 128, 208, 255, 255, 128, 129, 255, 150, 246

Bytes ranges: 192, 15, 1, 184, 5, 136, 6, 13, 1, 5, 5, 255, 3, 228, 135, 28, 232, 128, 208, 255, 255, 128, 129, 255, 150, 96

Tilings: [4, 1, 1, 4, 1, 3, 1, 1, 1, 1, 1, 5, 1, 5, 3, 1, 5, 3, 5, 5, 5, 3, 3, 5, 3, 2]


### Check the dimensions of the representation spaces

In [22]:
for representation_method in ['vector', 'tiling', 'one_hot_encoding', 'mixed']:
    environment.representation_method = representation_method
    print("{}: {}".format(representation_method, environment.get_input_dimension()))

(26,)
(730,)
(3074,)
(693,)


# Compare the current activated features and those from a loaded model

In [10]:
environment = env.OverSimplifiedEnvironment(
    environment=gym.make('Breakout-ram-v0'), 
    agent=None,
    verbose=True,
    max_steps=300,
    capacity=5000,
    representation_method='mixed',
    mixed_threshold=10,
    max_tiling=5
)
input_dimension = environment.get_input_dimension()
n_actions = environment.n_actions

model = linears.Linear(input_dimension=input_dimension, n_actions=n_actions)
optimiser = torch.optim.Adam(model.parameters(), lr=.001)
agent = agents.DQNAgent(model, optimiser, gamma=.999, temperature=10, algorithm='expsarsa', n_actions=n_actions)

environment.agent = agent

/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [11]:
print(environment.agent.q(environment.state_representation(environment.environment.reset())))
model.load_state_dict(torch.load('../saved/mixed_linear/state_dict.pth'))
agent.commit()
print(environment.agent.q(environment.state_representation(environment.environment.reset())))

[0.1980581  0.31158602 0.10360365]
[1.0813041  0.66803575 0.6778667 ]


In [16]:
activated_bytes = environment.features
min_bytes = environment.min_range
max_bytes = environment.max_range
bytes_range = max_bytes - min_bytes
tilings = environment.tilings

In [17]:
print(("{} activated bytes (coded on {} integers in total): " + ", ".join([str(x) for x in activated_bytes])).format(len(activated_bytes), len(activated_bytes) + sum(bytes_range)))
print()
print("Min bytes: " + ", ".join([str(x) for x in min_bytes]))
print()
print("Max bytes: " + ", ".join([str(x) for x in max_bytes]))
print()
print("Bytes ranges: " + ", ".join([str(x) for x in bytes_range]))
print()
print("Tilings: {}".format(tilings))


37 activated bytes (coded on 4126 integers in total): 0, 12, 18, 19, 24, 25, 30, 49, 57, 70, 71, 72, 74, 75, 77, 84, 86, 90, 91, 94, 95, 96, 99, 100, 101, 103, 105, 106, 107, 119, 121, 122, 102, 104, 109, 6, 31

Min bytes: 15, 207, 207, 243, 15, 252, 0, 225, 4, 0, 1, 55, 2, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150, 0, 0, 0, 63, 0

Max bytes: 63, 255, 255, 255, 255, 255, 192, 240, 5, 184, 6, 191, 8, 13, 1, 5, 25, 255, 3, 228, 135, 28, 232, 128, 208, 255, 255, 128, 129, 255, 150, 246, 128, 128, 2, 255, 192

Bytes ranges: 48, 48, 48, 12, 240, 3, 192, 15, 1, 184, 5, 136, 6, 13, 1, 5, 5, 255, 3, 228, 135, 28, 232, 128, 208, 255, 255, 128, 129, 255, 150, 96, 128, 128, 2, 192, 192

Tilings: [1, 1, 1, 1, 5, 1, 4, 1, 1, 4, 1, 3, 1, 1, 1, 1, 1, 5, 1, 5, 3, 1, 5, 3, 5, 5, 5, 3, 3, 5, 3, 2, 3, 3, 1, 4, 4]


In [12]:
m, M = environment.bytes_evolution_range(n_episodes_exploration=500, n_episodes_evaluation=100)

activated_bytes = np.nonzero(M - m)[0]
min_bytes = m[activated_bytes]
max_bytes = M[activated_bytes]
bytes_range = max_bytes - min_bytes

In [13]:
print(("{} activated bytes (coded on {} integers in total): " + ", ".join([str(x) for x in activated_bytes])).format(len(activated_bytes), len(activated_bytes) + sum(bytes_range)))
print()
print("Min bytes: " + ", ".join([str(x) for x in min_bytes]))
print()
print("Max bytes: " + ", ".join([str(x) for x in max_bytes]))
print()
print("Bytes ranges: " + ", ".join([str(x) for x in bytes_range]))


36 activated bytes (coded on 4411 integers in total): 0, 6, 12, 18, 19, 24, 30, 31, 49, 57, 70, 71, 72, 74, 75, 77, 84, 86, 90, 91, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 119, 121, 122

Min bytes: 15, 63, 63, 63, 252, 15, 0, 0, 225, 4, 0, 1, 55, 2, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 150

Max bytes: 63, 255, 255, 255, 255, 255, 192, 192, 240, 5, 184, 6, 191, 8, 13, 3, 15, 25, 255, 3, 226, 135, 28, 230, 128, 208, 128, 255, 128, 255, 128, 131, 2, 255, 150, 246

Bytes ranges: 48, 192, 192, 192, 3, 240, 192, 192, 15, 1, 184, 5, 136, 6, 13, 3, 15, 5, 255, 3, 226, 135, 28, 230, 128, 208, 128, 255, 128, 255, 128, 131, 2, 255, 150, 96


In [14]:
new_bytes = [x for x in activated_bytes if x not in environment.features]
new_min_bytes = m[new_bytes]
new_max_bytes = M[new_bytes]
new_bytes_range = new_max_bytes - new_min_bytes

In [15]:
print(("{} new bytes: " + ", ".join([str(x) for x in new_bytes])).format(len(new_bytes)))
print()
print("Min bytes: " + ", ".join([str(x) for x in new_min_bytes]))
print()
print("Max bytes: " + ", ".join([str(x) for x in new_max_bytes]))
print()
print("Bytes ranges: " + ", ".join([str(x) for x in new_bytes_range]))


0 new bytes: 

Min bytes: 

Max bytes: 

Bytes ranges: 
